In [ ]:
# this notebook is testing the TA lib as its being built from notes
# this TA lib will be a complete rewrite in python. By alphagriffin.

In [1]:
%matplotlib inline
# generic
import os, sys, time, datetime, collections, re, random, asyncio
from IPython.display import display
from timeit import default_timer as timer
from tqdm import tqdm, trange
import numpy as np
import pandas as pd

import ag.bittensor.utils.options as options
import ag.bittensor.ai.make_data as make_data


import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib.dates as mdates

from sklearn.neural_network import MLPClassifier
from sklearn import datasets

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing.data import QuantileTransformer

import ag.bittensor.utils.talib as talib

In [2]:
# Funcs
def get_all_data():
    all_data = {}
    while True:
        filename = datasmith.next_filename
        if filename is None: break
        datasmith.dataframe = filename
        all_data[filename[:-4]] = datasmith.dataframe
    return all_data

def get_one_data():
    filename = datasmith.random_filename
    datasmith.dataframe = filename
    return datasmith.dataframe

def unzero(df):
    df = df.replace(0,'NaN')
    df = df.dropna(how='all',axis=0)
    df = df.replace('NaN', 0)
    df.len = len(df)
    return df

def get_change(df, period=60):  
    # not really using this... just keeping around for notes
    return unzero(df.pct_change(periods = period))

In [3]:
# Globals
datasmith = make_data.MakeData()
TA = talib.TALib()

start = timer()
get_all_data = False
if get_all_data:
    mega_set = get_all_data()
    btc_pairs = [x for x in mega_set if 'BTC' in x]
    btc_set = {}
    for i in mega_set:
        if i in btc_pairs:
            btc_set[i] = mega_set[i]
    highvol_set = {}
    volumes = []
    pairs = []
    for i in btc_set:
        btc_vol = btc_set[i]['baseVolume'][0] * btc_set[i]['last'][0]
        if btc_vol > 100:
            highvol_set[i] = btc_set[i]
            volumes.append(btc_vol)
            pairs.append(i)
    print('Picking the right data took {:.2f} secs'.format(timer()-start))
    print('Working with {} pairs.'.format(len(highvol_set)))
else:
    df = get_one_data()
    print('Got {} | Took: {:.2f} secs'.format(datasmith.pair, timer()-start))

Got DASH_BTC | Took: 0.01 secs


In [4]:
# moving average
_, df = TA.MA(df, 30)
_, df = TA.MA(df, 30, 'baseVolume')
# expontential moving average
_, df = TA.EMA(df, 30)
_, df = TA.EMA(df, 30, 'baseVolume')
# momentum
_, df = TA.MOM(df, 30)
_, df = TA.MOM(df, 30, 'baseVolume')
# rate of change
_, df = TA.ROC(df, 30)
_, df = TA.ROC(df, 30, 'baseVolume')

In [5]:
print(df.tail(2))

          timestamp    high       low    last    change   baseVolume  \
4971  1519295722823  0.0649  0.061565  0.0625 -0.021144  2664.580869   
4972  1519295843001  0.0649  0.061565  0.0620 -0.029732  2664.947175   

      MA_last_30  MA_baseVolume_30  EMA_last_30  EMA_baseVolume_30  \
4971    0.062560       2676.424197     0.062504        2678.125794   
4972    0.062543       2675.466665     0.062471        2677.275560   

      Momentum_last_30  Momentum_baseVolume_30  RateOfChange_last_30  \
4971      3.000000e-08              -26.555091              0.000800   
4972     -4.998500e-04              -28.725981             -0.007997   

      RateOfChange_baseVolume_30  
4971                   -0.018728  
4972                   -0.009722  


In [15]:
mom = df['Momentum_last_30'].mean()
rock = df['RateOfChange_last_30'].mean()

In [23]:
print('Buy/Hold {} | {:.8f} BTC or {:.2}% per 30 mins over last {:.1f} days'.format(datasmith.pair, mom, rock, len(df)/60/24))

Buy/Hold DASH_BTC | -0.00002019 BTC or -0.0003% per 30 mins over last 3.5 days
